In [ ]:

# Set up the notebook
%pprint
import sys
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))

In [ ]:

from FRVRS import (
    nu, display
)


# Add a Voice Capture Sentiment Score

In [ ]:

# Add a voice capture sentiment score
new_column_name = 'voice_capture_sentiment_score'
if (new_column_name not in frvrs_logs_df.columns):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    try: sid = SentimentIntensityAnalyzer()
    except LookupError as e:
        print(str(e).strip())
        import nltk
        nltk.download('vader_lexicon')
        sid = SentimentIntensityAnalyzer()
    mask_series = frvrs_logs_df.voice_capture_message.isnull()
    for row_index, row_series in frvrs_logs_df[~mask_series].iterrows():
        voice_capture_message = '\n' + row_series.voice_capture_message
        frvrs_logs_df.loc[row_index, new_column_name] = sid.polarity_scores(voice_capture_message)['compound']
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    
    print(frvrs_logs_df.shape) # (829116, 114)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}).sort_values('record_count', ascending=False).head(5))